**Ateema AI video generation code**





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **1. Text-to-Speech system package install**

In [ ]:
# Tortoise

!git clone https://github.com/DjKesu/tortoise-tts-fast-cloning.git
%cd tortoise-tts-fast-cloning
!pip3 install -r requirements.txt --no-deps
!pip3 install -e .
!pip3 install git+https://github.com/152334H/BigVGAN.git

fatal: destination path 'tortoise-tts-fast-cloning' already exists and is not an empty directory.
/content/tortoise-tts-fast-cloning
Ignoring backports-zoneinfo: markers 'python_version >= "3.8" and python_version < "3.9"' don't match your environment
ERROR: Can't verify hashes for these requirements because we don't have a way to hash version control repositories:
    bigvgan@ git+https://github.com/152334H/BigVGAN.git@HEAD from git+https://github.com/152334H/BigVGAN.git@HEAD (from -r requirements.txt (line 38))
Obtaining file:///content/tortoise-tts-fast-cloning
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Cloning https://github.com/152334H/BigVGAN.git to /tmp/pip-install-vywpd3wu/bigvgan_9e35a3eedbb64ec29235ecd273115a28
  Running command git clone --filter=blob:none --quiet https://github.com/152334H/BigVGAN.git /tmp/pip

# **2. Peronalization - avatar image and video background image**

**Download and prepare personalized images to be used in video generation, images will be loaded in the content/ section of notebook**

In [ ]:
# Download image
!wget https://raw.githubusercontent.com/alexacheui/ateema_capstone/main/video_source/avatar_img.png
!wget https://raw.githubusercontent.com/alexacheui/ateema_capstone/main/video_source/ateema_background_2.png

--2024-08-18 15:13:22--  https://raw.githubusercontent.com/alexacheui/ateema_capstone/main/video_source/avatar_img.png
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 350188 (342K) [image/png]
Saving to: ‘avatar_img.png’

avatar_img.png      100%[===================>] 341.98K  --.-KB/s    in 0.004s  

2024-08-18 15:13:22 (82.0 MB/s) - ‘avatar_img.png’ saved [350188/350188]

--2024-08-18 15:13:22--  https://raw.githubusercontent.com/alexacheui/ateema_capstone/main/video_source/ateema_background_2.png
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response..

In [ ]:
!pwd

/content/tortoise-tts-fast-cloning


# **3. Installation of avatar generation packages**

In [ ]:
!pip uninstall transformers -y -q
!pip install transformers==4.29.2 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 97.6 MB/s eta 0:00:00


In [ ]:
# SadTalker

!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.8 2
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.9 1
!python --version
!apt-get update
!apt install software-properties-common
!sudo dpkg --remove --force-remove-reinstreq python3-pip python3-setuptools python3-wheel
!apt-get install python3-pip

print('Git clone project and install requirements...')
!git clone https://github.com/cedro3/SadTalker.git &> /dev/null
%cd SadTalker
!export PYTHONPATH=/content/SadTalker:$PYTHONPATH
!python3 -m pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
!apt update
!apt install ffmpeg &> /dev/null
!python3 -m pip install -r requirements.txt

print('Download pre-trained models...')
!rm -rf checkpoints
!bash scripts/download_models.sh

update-alternatives: error: alternative path /usr/bin/python3.8 doesn't exist
update-alternatives: error: alternative path /usr/bin/python3.9 doesn't exist
Python 3.10.12
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configu

# **4. Installation of video production packages**

In [ ]:
# Stable Diffusion
!pip install diffusers requests -q

In [ ]:
# video Production

!pip install moviepy -q
!pip install google-auth-oauthlib google-auth-httplib2 google-api-python-client -q

# **5. Packages import**

In [ ]:
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F
import re
import IPython
from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices
import os
from IPython.display import display, Video
import locale
import shutil
import logging
import requests
from PIL import Image, ImageDraw, ImageFont
from io import BytesIO
import pandas as pd
from diffusers import StableDiffusionPipeline
import moviepy.editor as mp
import numpy as np
from google.colab import drive
from google.colab import auth
from googleapiclient.discovery import build
import shutil
import cv2

#**6. Video genertation pipeline code**

In [ ]:
import re
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F
import IPython
from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices
import os
from IPython.display import display, Video, Audio
import locale
import shutil
import logging
import requests
from PIL import Image, ImageDraw, ImageFont
from io import BytesIO
import pandas as pd
from diffusers import StableDiffusionPipeline
import moviepy.editor as mp
import numpy as np
from google.colab import drive
from google.colab import auth
from googleapiclient.discovery import build

class RecommendationProcessor:
    def __init__(self):
        self.logger = self.setup_logger()
        self.tts = TextToSpeech()
        self.model = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16).to("cuda" if torch.cuda.is_available() else "cpu")

    def setup_logger(self):
        logger = logging.getLogger('RecommendationProcessor')
        logger.setLevel(logging.INFO)
        handler = logging.StreamHandler()
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        handler.setFormatter(formatter)
        logger.addHandler(handler)
        return logger

    def dynamically_modify_recommendation(self, text):
        pattern_remove = r"(AI tour guide\.\s)(.*?)(I'd be happy to recommend some interesting places to visit\.\s)"
        modified_text = re.sub(pattern_remove, r"\1\3", text)
        return modified_text

    def replace_numbers_with_natural_transitions(self, text):
        transitions = [
            "First up,",
            "Following that,",
            "Another great option is,",
            "You might also like,",
            "For something different,",
            "If you're looking for more,",
            "Don't miss,",
            "A must-visit is,",
            "Lastly,"
        ]
        pattern = r'\b(\d+)\.\s'
        transition_index = 0

        def replace_match(match):
            nonlocal transition_index
            transition = transitions[transition_index % len(transitions)]
            transition_index += 1
            return f"{transition} {match.group(0)[3:]}"  # Skip the "1. ", "2. ", etc.

        result = re.sub(pattern, replace_match, text)
        return result

    def clean_text(self, text):
        self.logger.info("Cleaning text")
        cleaned = re.sub(r'\*+', '', text)
        return cleaned.strip()

    def process_in_chunks(self, text, chunk_size=500):
        self.logger.info("Processing text in chunks")
        sentences = re.split(r'(?<=[.!?])\s+', text)
        chunks = []
        current_chunk = []

        for sentence in sentences:
            if len(' '.join(current_chunk)) + len(sentence) > chunk_size and current_chunk:
                chunks.append(' '.join(current_chunk))
                current_chunk = []
            current_chunk.append(sentence)

        if current_chunk:
            chunks.append(' '.join(current_chunk))

        return chunks

    def generate_audio(self, recommendation):
        self.logger.info("Generating audio")
        clean_recommendation = self.clean_text(recommendation)
        chunks = self.process_in_chunks(clean_recommendation)
        self.logger.info(f"Number of chunks: {len(chunks)}")

        preset = "standard"
        voice = 'emma'
        voice_samples, conditioning_latents = load_voice(voice)

        all_audio = []

        for i, chunk in enumerate(chunks):
            self.logger.info(f"Processing chunk {i+1}/{len(chunks)}")
            audio = self.tts.tts_with_preset(chunk, voice_samples=voice_samples, conditioning_latents=conditioning_latents, preset=preset)
            all_audio.append(audio.cpu().squeeze(0))

        combined_audio = torch.cat([a.view(1, -1) for a in all_audio], dim=1)
        os.chdir('/content')
        torchaudio.save('generated_combined.wav', combined_audio, 24000)
        self.logger.info(f"Total audio length: {combined_audio.shape[1]} frames")
        return '/content/generated_combined.wav'

    def create_avatar(self):
        self.logger.info("Creating avatar")
        os.chdir('/content')
        folder_path = 'avatar_sadtalker'

        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            self.logger.info(f"Folder '{folder_path}' created successfully.")
        else:
            self.logger.info(f"Folder '{folder_path}' already exists.")

        os.chdir('/content/SadTalker')
        source_image = '/content/avatar_img.png'
        driven_audio = '/content/generated_combined.wav'

        os.system(f"python3 inference.py --driven_audio {driven_audio} "
                  f"--source_image {source_image} "
                  f"--result_dir /content "
                  f"--enhancer gfpgan")

    def move_and_rename_latest_video(self, source_dir, target_dir, new_name='avatar_video.mp4'):
        self.logger.info("Moving and renaming latest video")
        video_files = [f for f in os.listdir(source_dir) if f.startswith("20") and f.endswith(".mp4")]

        if not video_files:
            self.logger.warning("No matching video files found.")
            return None

        latest_video = max(video_files, key=lambda f: os.path.getmtime(os.path.join(source_dir, f)))

        source_path = os.path.join(source_dir, latest_video)
        target_path = os.path.join(target_dir, new_name)

        if not os.path.exists(target_dir):
            os.makedirs(target_dir)

        try:
            shutil.move(source_path, target_path)
            self.logger.info(f"Video moved and renamed to: {target_path}")
            return target_path
        except Exception as e:
            self.logger.error(f"Error moving and renaming video: {e}")
            return None

    def process_images(self, url_information):
        self.logger.info("Processing images")
        os.chdir('/content')
        if not os.path.exists('video_image'):
            os.makedirs('video_image')
            self.logger.info("Created folder: video_image")

        filtered_data = {"Title": [], "URL": []}
        for entry in url_information:
            if entry["Match"] == "Yes" and entry["URL"] != "No URL available":
                filtered_data["Title"].append(entry["Title"])
                filtered_data["URL"].append(entry["URL"])

        df = pd.DataFrame(filtered_data)

        if df.empty:
            self.logger.warning("No valid URLs found. Folder created but no images downloaded.")
            return

        for index, row in df.iterrows():
            title = row['Title'].replace(' ', '_')
            url = row['URL']
            image_path = os.path.join('video_image', f"{title}.png")

            try:
                response = requests.get(url)
                response.raise_for_status()

                with open(image_path, 'wb') as file:
                    file.write(response.content)
                self.logger.info(f"Downloaded: {image_path}")

                img = Image.open(image_path).convert("RGB")
                png_filename = os.path.splitext(os.path.basename(image_path))[0] + ".png"
                png_path = os.path.join('video_image', png_filename)
                img.save(png_path, "PNG")
                self.logger.info(f"Converted and saved: {png_path}")
            except requests.RequestException as e:
                self.logger.error(f"Failed to download {url}: {e}")
            except Exception as e:
                self.logger.error(f"Error processing {url}: {e}")

    def generate_or_download_image_with_label(self, entry):
        self.logger.info(f"Generating or downloading image for: {entry['Title']}")
        place = entry["Title"]

        if entry["Match"] == "Yes" and entry["URL"] != "No URL available":
            response = requests.get(entry["URL"])
            image = Image.open(BytesIO(response.content)).convert("RGB")
        else:
            with torch.no_grad():
                image = self.model(f"{place} is either a restaurant or event in Chicago").images[0]
            image = image.convert("RGB")

        draw = ImageDraw.Draw(image)

        try:
            font_path = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"
            font_size = 80
            font = ImageFont.truetype(font_path, font_size)
        except IOError:
            font = ImageFont.load_default()
            font_size = 40

        max_width = image.width - 20
        text_bbox = draw.textbbox((0, 0), place, font=font)
        text_width = text_bbox[2] - text_bbox[0]

        while text_width > max_width and font_size > 10:
            font_size -= 5
            font = ImageFont.truetype(font_path, font_size)
            text_bbox = draw.textbbox((0, 0), place, font=font)
            text_width = text_bbox[2] - text_bbox[0]

        text_height = text_bbox[3] - text_bbox[1]
        text_position = ((image.width - text_width) / 2, 20)

        rectangle_padding = 10
        rect_position = [
            text_position[0] - rectangle_padding,
            text_position[1] - rectangle_padding,
            text_position[0] + text_width + rectangle_padding,
            text_position[1] + text_height + rectangle_padding
        ]
        draw.rectangle(rect_position, fill="black")

        draw.text(text_position, place, font=font, fill="white")

        return image

    def process_all_images(self, image_url):
        self.logger.info("Processing all images")
        os.chdir('/content')
        if not os.path.exists("video_image"):
            os.makedirs("video_image")

        for entry in image_url:
            image = self.generate_or_download_image_with_label(entry)
            image_path = os.path.join("video_image", f"{entry['Title'].replace(' ', '_')}.png")
            image.save(image_path)
            self.logger.info(f"Image processed and saved for: {entry['Title']}")

        self.logger.info("All images have been processed and saved in the 'video_image' folder.")

    def extract_places_and_timestamps(self, recommendation, audio_duration):
        self.logger.info("Extracting places and timestamps")

        # Define transitions that introduce the places
        transitions = [
            "First up,",
            "Following that,",
            "Another great option is,",
            "You might also like,",
            "For something different,",
            "If you're looking for more,",
            "Don't miss,",
            "A must-visit is,",
            "Lastly,"
        ]

        # Extract places from the text that follow the transitions
        places = []
        for transition in transitions:
            pattern = re.compile(rf"{transition}\s+\*\*(.*?)\*\*")
            matches = pattern.findall(recommendation)
            places.extend(matches)

        places = list(set(places))  # Remove duplicates

        timestamps = {}
        words = re.findall(r'\w+', recommendation)  # Tokenize words without punctuation

        def find_best_start_time(place_words):
            total_place_words = len(place_words)
            best_start_time = None
            best_match_count = 0

            for i in range(len(words) - total_place_words + 1):
                match_count = sum(1 for j in range(total_place_words) if place_words[j].lower() in words[i + j].lower())

                if match_count > best_match_count:
                    best_match_count = match_count
                    start_index = i
                    best_start_time = ((start_index / len(words)) * audio_duration)

            self.logger.info(f"Place: {' '.join(place_words)} | Match Count: {best_match_count} | Start Time: {best_start_time}")
            return best_start_time

        for place in places:
            place_words = place.split()
            best_start_time = find_best_start_time(place_words)

            if best_start_time is not None:
                timestamps[place] = best_start_time

        self.logger.info(f"Extracted places and timestamps: {timestamps}")
        return timestamps

    def create_synchronized_video(self, audio_path, image_paths, timestamps, output_path, avatar_video_path, audio_duration, initial_background_path):
        self.logger.info("Creating synchronized video")
        avatar_clip = mp.VideoFileClip(avatar_video_path)
        audio_clip = mp.AudioFileClip(audio_path)

        background_clip = self.convert_to_rgb(initial_background_path).set_duration(audio_duration)
        background_clip = background_clip.set_position(("center", "center")).resize(height=avatar_clip.h)

        final_clips = [background_clip]

        sorted_timestamps = sorted(timestamps.items(), key=lambda x: x[1])
        for i, (place, start_time) in enumerate(sorted_timestamps):
            if place in image_paths:
                end_time = sorted_timestamps[i + 1][1] if i + 1 < len(sorted_timestamps) else audio_duration
                img_clip = self.convert_to_rgb(image_paths[place]).set_start(start_time).set_duration(end_time - start_time)
                img_clip = img_clip.resize((250, 250))
                x_position = background_clip.w - img_clip.w - 20
                y_position = background_clip.h - img_clip.h - 200
                img_clip = img_clip.set_position((x_position, y_position))
                final_clips.append(img_clip)

                self.logger.info(f"Added image: {place} at timestamp: {start_time} for duration: {end_time - start_time}")

        avatar_clip = avatar_clip.resize((110, 110))
        x_position = background_clip.w - avatar_clip.w - 160
        y_position = background_clip.h - avatar_clip.h - 10
        avatar_clip = avatar_clip.set_position((x_position, y_position))
        final_clips.append(avatar_clip)

        final_clip = mp.CompositeVideoClip(final_clips).set_audio(audio_clip)
        final_clip.write_videofile(output_path, codec="libx264", audio_codec="aac")

        self.logger.info(f"Video created successfully at {output_path}")

    def convert_to_rgb(self, image_path):
        image = Image.open(image_path)
        if image.mode != 'RGB':
            image = image.convert('RGB')
        return mp.ImageClip(np.array(image))

    def upload_to_drive(self, source_path, destination_path):
        self.logger.info("Uploading to Google Drive")
        drive.mount('/content/drive')
        shutil.copy(source_path, destination_path)

        auth.authenticate_user()
        drive_service = build('drive', 'v3')

        file_name = os.path.basename(destination_path)
        results = drive_service.files().list(q=f"name='{file_name}'", spaces='drive', fields='files(id, name)').execute()
        files = results.get('files', [])

        if not files:
            self.logger.warning(f"No file named '{file_name}' found in Google Drive.")
            return None
        else:
            file_id = files[0]['id']
            self.logger.info(f"File ID: {file_id}")

            share_link = self.create_sharable_link(drive_service, file_id)
            self.logger.info(f"Sharable link: {share_link}")
            return share_link

    def create_sharable_link(self, drive_service, file_id):
        request_body = {
            'role': 'reader',
            'type': 'anyone'
        }
        response_permission = drive_service.permissions().create(fileId=file_id, body=request_body).execute()
        response_share_link = drive_service.files().get(fileId=file_id, fields='webViewLink').execute()
        return response_share_link.get('webViewLink')

    def process_recommendation(self, recommendation, image_url):
        self.logger.info("Processing recommendation")

        # Step 1: Dynamically modify the text and replace numbers with transitions
        modified_recommendation = self.dynamically_modify_recommendation(recommendation)
        final_recommendation = self.replace_numbers_with_natural_transitions(modified_recommendation)

        # Print the final recommendation to debug the modifications
        self.logger.info(f"Modified Recommendation Text: {final_recommendation}")

        # Step 2: Generate audio from the final recommendation
        audio_path = self.generate_audio(final_recommendation)

        # Step 3: Create the avatar video
        self.create_avatar()
        source_dir = '/content'
        target_dir = '/content/avatar_sadtalker'
        generated_video = self.move_and_rename_latest_video(source_dir, target_dir)

        if generated_video:
            display(Video(generated_video, embed=True, width=400, height=400))
        else:
            self.logger.warning("No video file moved or renamed.")

        # Step 4: Process images based on provided URLs or generate new ones
        self.process_images(image_url)
        self.process_all_images(image_url)

        # Step 5: Extract places and timestamps for synchronization
        audio_clip = mp.AudioFileClip(audio_path)
        audio_duration = audio_clip.duration
        timestamps = self.extract_places_and_timestamps(final_recommendation, audio_duration)

        # Step 6: Create the synchronized video with all the elements
        audio_filename = audio_path
        avatar_video_path = "/content/avatar_sadtalker/avatar_video.mp4"
        initial_background_path = "/content/ateema_background_2.png"
        output_video_path = "/content/final_output.mp4"

        image_paths = {place: f"/content/video_image/{place.replace(' ', '_')}.png" for place in timestamps.keys()}

        self.create_synchronized_video(audio_filename, image_paths, timestamps, output_video_path,
                                       avatar_video_path, audio_duration, initial_background_path)

        # Display the final video to the user
        display(Video(output_video_path, embed=True, width=600, height=400))

        # Step 7: Upload the final video to Google Drive
        source_path = '/content/final_output.mp4'
        destination_path = '/content/drive/My Drive/final_output.mp4'
        share_url = self.upload_to_drive(source_path, destination_path)

        return share_url

    def run(self, recommendation, image_url):
        self.logger.info("Starting recommendation processing")
        locale.getpreferredencoding = lambda: "UTF-8"
        share_url = self.process_recommendation(recommendation, image_url)
        self.logger.info("Recommendation processing completed")

        return share_url


# **7.  Generating video triggering app to send requests to control system**

In [ ]:
!pip install flask pyngrok --quiet

In [ ]:
pip install pyngrok

In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
from datetime import datetime
from google.colab import userdata
from pyngrok import ngrok
import json
from threading import Thread
import uuid

def process_input(input_data):
    # If input_data is already a dictionary, no need to parse it
    if isinstance(input_data, dict):
        data = input_data
    else:
        # If it's a string, parse it as JSON
        try:
            data = json.loads(input_data)
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON: {e}")
            return None, None

    print(data)

    # Extract the recommendation as a string
    recommendation = data['recommendation']

    # Extract the image_url information
    image_url = data['url_information']

    return recommendation, image_url

processor = RecommendationProcessor()

app = Flask(__name__)

jobs = {}

@app.route('/webhook', methods=['POST'])
def typeform_webhook():
    try:
        request_data = request.get_json()
        if request_data is None:
            return jsonify({'message': 'No JSON data received'}), 400

        # Generate a unique job ID
        job_id = str(uuid.uuid4())

        # Store initial job status
        jobs[job_id] = {'status': 'processing', 'result': None}

        # Start processing in a background thread
        Thread(target=process_video, args=(job_id, request_data)).start()

        # Immediately return a success response with the job ID
        return jsonify({'message': 'Processing started', 'job_id': job_id}), 202
    except Exception as e:
        print(f"Error processing request: {str(e)}")
        return jsonify({'message': 'Error processing request', 'error': str(e)}), 500

def process_video(job_id, input_data):
    try:
        recommendation, image_url = process_input(input_data)
        video_url = processor.run(recommendation, image_url)

        # Store the result
        jobs[job_id] = {'status': 'completed', 'result': video_url}
        print(f"Video processing completed: {video_url}")
    except Exception as e:
        jobs[job_id] = {'status': 'failed', 'result': str(e)}
        print(f"Error in background processing: {str(e)}")

@app.route('/status/<job_id>', methods=['GET'])
def check_status(job_id):
    job = jobs.get(job_id)
    print(job_id)
    if not job:
        return jsonify({'message': 'Job not found'}), 404
    return jsonify(job)

if __name__ == '__main__':
    ngrok.set_auth_token(userdata.get('ngrok_key'))
    ngrok_tunnel = ngrok.connect(5000)
    print('Public URL:', ngrok_tunnel.public_url)
    app.run(port=5000)

#**8. Manual approach of generating video**

**1. Place the output of reccomendation section as provided by the reccomendation system**

In [ ]:
recommendation = "Hello Jessica! I'm Ateema, your AI tour guide. You're planning a trip to Chicago from September 1st to September 10th with a group of three people aged 21-30. Your interests include dining (Outdoor, Food), beverages (Rooftop, Cocktails), entertainment (Concerts, Comedy), cultural activities (Art, Architecture), outdoor activities (River, Boat), educational activities (Landmarks, Art), and shopping (Shopping, Local). I'd be happy to recommend some interesting places to visit. Here are my suggestions: **Dining:** 1. **The Gage**: This upscale restaurant offers a seasonal menu with a focus on local ingredients and stunning views of Millennium Park; 2. **Girl & the Goat**: A Mediterranean-inspired restaurant from Top Chef winner Stephanie Izard, offering creative small plates and craft cocktails. **Beverages:** 1. **The Roof at The Wit Hotel**: Enjoy rooftop cocktails and take in the breathtaking views of the city; 2. **Big Star Tacos**: Sip on a craft beer or cocktail while enjoying delicious tacos and people-watching. **Entertainment:** 1. **The Vic Theatre**: Catch a live concert or comedy show at this historic theater; 2. **Second City**: Laugh out loud with improv comedy from the famous Second City troupe. **Cultural Activities:** 1. **The Art Institute of Chicago**: Explore an impressive collection of Impressionist, Post-Impressionist, and American art; 2. **The Chicago Architecture River Cruise**: Learn about the city's rich architectural history while cruising along the Chicago River. **Outdoor Activities:** 1. **The Centennial Wheel at Navy Pier**: Take in panoramic views of Lake Michigan and the city skyline; 2. **The 606**: Explore this elevated park built on an old rail line, featuring public art installations and great city views. **Educational Activities:** 1. **The Field Museum**: Discover ancient cultures and natural history exhibits at one of the largest natural history museums in the world; 2. **The Chicago Cultural Center**: Learn about the city's rich cultural heritage through exhibitions, performances, and tours. **Shopping:** 1. **The Magnificent Mile**: Shop along this famous stretch of Michigan Avenue, featuring high-end boutiques and iconic landmarks; 2. **Wicker Park**: Explore this trendy neighborhood filled with unique shops, galleries, and restaurants. I hope you enjoy your trip to Chicago! Have a great time exploring this amazing city."

**2. Place image urls as provided from the reccomendations system**

In [ ]:
image_url =[
    {
      "Title": "The Gage",
      "Match": "No",
      "URL": "No URL available"
    },
    {
      "Title": "Girl & the Goat",
      "Match": "No",
      "URL": "No URL available"
    },
    {
      "Title": "The Roof at The Wit Hotel",
      "Match": "No",
      "URL": "No URL available"
    },
    {
      "Title": "Big Star Tacos",
      "Match": "No",
      "URL": "No URL available"
    },
    {
      "Title": "The Vic Theatre",
      "Match": "No",
      "URL": "No URL available"
    },
    {
      "Title": "Second City",
      "Match": "Yes",
      "URL": "No URL available"
    },
    {
      "Title": "The Art Institute of Chicago",
      "Match": "Yes",
      "URL": "https://cdn.choosechicago.com/uploads/2019/08/07195792-e1566185494402-900x530.jpg"
    },
    {
      "Title": "The Chicago Architecture River Cruise",
      "Match": "No",
      "URL": "No URL available"
    },
    {
      "Title": "The Centennial Wheel at Navy Pier",
      "Match": "No",
      "URL": "No URL available"
    },
    {
      "Title": "The 606",
      "Match": "Yes",
      "URL": "No URL available"
    },
    {
      "Title": "The Field Museum",
      "Match": "Yes",
      "URL": "No URL available"
    },
    {
      "Title": "The Chicago Cultural Center",
      "Match": "Yes",
      "URL": "https://cdn.choosechicago.com/uploads/2019/06/8add71f9_9052_45a9_aeb5_694731edd746_52bed905-6baa-4b77-ad49-fec9e2154e62-1.jpg"
    },
    {
      "Title": "The Magnificent Mile",
      "Match": "Yes",
      "URL": "No URL available"
    },
    {
      "Title": "Wicker Park",
      "Match": "No",
      "URL": "No URL available"
    }
  ]

**3. Run the video generation pipeline code**

**4. Run the manual video generation trigger which will generate the video without making calls to other system (control center)**

In [ ]:
if __name__ == '__main__':
    processor = RecommendationProcessor()
    video_url = processor.run(recommendation, image_url)
    print(f"The video is available at: {video_url}")

# **9. Archive - sample of jason file of reccomendation output**

In [ ]:
input_data = {
  "recommendation": "{ \"Hello Jessica. I'm Ateema, your AI tour guide. You're planning a trip to Chicago with a group of 42 people aged 20-30 and will be visiting from February 6th to February 6th. I've got you covered with some fantastic recommendations for dining, beverages, entertainment, cultural activities, outdoor activities, educational activities, and shopping. Here's what I've got in store for you: **Dining:** 1. **Girl & the Goat**: This Mediterranean-inspired restaurant from Top Chef winner Stephanie Izard is perfect for a group of foodies, offering a variety of small plates and wood-fired dishes. 2. **Portillo's**: For a casual dining experience, head to Portillo's, where you can enjoy classic Chicago-style hot dogs, Italian beef sandwiches, and more. **Beverages:** 1. **The Violet Hour**: This swanky cocktail bar is a must-visit for any spirits enthusiast, with expertly crafted drinks and a sophisticated atmosphere. 2. **Big Jones**: For a unique drinking experience, visit Big Jones, which offers craft cocktails and a menu of Southern-inspired small plates. **Entertainment:** 1. **Second City**: Catch a comedy show at Second City, one of the most renowned improv institutions in the world. 2. **The Chicago Theatre**: This historic theater hosts a variety of performances, from concerts to plays, making it an excellent choice for your group. **Cultural Activities:** 1. **The Art Institute of Chicago**: Explore the impressive collection at this world-renowned museum, featuring works by famous artists like Monet and Picasso. 2. **The Field Museum**: Discover ancient cultures and natural history at this iconic museum, home to Sue, the largest T-Rex skeleton in the world. **Outdoor Activities:** 1. **Millennium Park**: Take a stroll through this beautiful park, featuring iconic landmarks like the \"\t             \n                            \t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n                            \t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t: \n                            \t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\" Cloud Gate sculpture. 2. **Navy Pier**: Enjoy the lakefront views and take a ride on the Centennial Wheel for panoramic views of the city. **Educational Activities:** 1. **The Chicago History Museum**: Learn about the city's rich history at this museum, featuring exhibits on everything from the Great Fire to the Civil Rights Movement. 2. **The Museum of Science and Industry**: Explore the wonders of science and technology at this interactive museum, with exhibits like a coal mine and a German submarine. **Shopping:** 1. **The Magnificent Mile**: Treat your group to some upscale shopping along this famous stretch of Michigan Avenue, featuring iconic department stores and boutiques. 2. **Wicker Park**: Explore the unique shops and galleries in this trendy neighborhood, offering everything from vintage clothing to handmade jewelry. I hope you enjoy your trip to Chicago! Have a great time exploring this amazing city.\" }",
  "url_information": [
    {
      "Title": "Girl & the Goat",
      "Match": "No",
      "URL": "No URL available"
    },
    {
      "Title": "Portillo's",
      "Match": "No",
      "URL": "No URL available"
    },
    {
      "Title": "The Violet Hour",
      "Match": "Yes",
      "URL": "No URL available"
    },
    {
      "Title": "Big Jones",
      "Match": "Yes",
      "URL": "No URL available"
    },
    {
      "Title": "Second City",
      "Match": "Yes",
      "URL": "No URL available"
    },
    {
      "Title": "The Chicago Theatre",
      "Match": "No",
      "URL": "No URL available"
    },
    {
      "Title": "The Art Institute of Chicago",
      "Match": "Yes",
      "URL": "https://cdn.choosechicago.com/uploads/2019/08/07195792-e1566185494402-900x530.jpg"
    },
    {
      "Title": "The Field Museum",
      "Match": "Yes",
      "URL": "No URL available"
    },
    {
      "Title": "Millennium Park",
      "Match": "Yes",
      "URL": "No URL available"
    },
    {
      "Title": "Navy Pier",
      "Match": "Yes",
      "URL": "https://cdn.choosechicago.com/uploads/2019/05/NAVY-PIER-79-900x601.jpg"
    },
    {
      "Title": "The Chicago History Museum",
      "Match": "Yes",
      "URL": "No URL available"
    },
    {
      "Title": "The Museum of Science and Industry",
      "Match": "Yes",
      "URL": "https://cdn.choosechicago.com/uploads/2019/06/ScienceStorms_Vortex-900x537.jpg"
    },
    {
      "Title": "The Magnificent Mile",
      "Match": "Yes",
      "URL": "No URL available"
    },
    {
      "Title": "Wicker Park",
      "Match": "No",
      "URL": "No URL available"
    }
  ]
}

In [ ]:
import json
def process_input(input_data):
    # If input_data is already a dictionary, no need to parse it
    if isinstance(input_data, dict):
        data = input_data
    else:
        # If it's a string, parse it as JSON
        data = json.loads(input_data)

    # Extract the recommendation
    recommendation = json.loads(data['recommendation'])

    # Extract the image_url information
    image_url = data['url_information']

    return recommendation, image_url

In [ ]:
recommendation, image_url = process_input(input_data)

In [ ]:
print(image_url)

[{'Title': 'Girl & the Goat', 'Match': 'No', 'URL': 'No URL available'}, {'Title': "Portillo's", 'Match': 'No', 'URL': 'No URL available'}, {'Title': 'The Violet Hour', 'Match': 'Yes', 'URL': 'No URL available'}, {'Title': 'Big Jones', 'Match': 'Yes', 'URL': 'No URL available'}, {'Title': 'Second City', 'Match': 'Yes', 'URL': 'No URL available'}, {'Title': 'The Chicago Theatre', 'Match': 'No', 'URL': 'No URL available'}, {'Title': 'The Art Institute of Chicago', 'Match': 'Yes', 'URL': 'https://cdn.choosechicago.com/uploads/2019/08/07195792-e1566185494402-900x530.jpg'}, {'Title': 'The Field Museum', 'Match': 'Yes', 'URL': 'No URL available'}, {'Title': 'Millennium Park', 'Match': 'Yes', 'URL': 'No URL available'}, {'Title': 'Navy Pier', 'Match': 'Yes', 'URL': 'https://cdn.choosechicago.com/uploads/2019/05/NAVY-PIER-79-900x601.jpg'}, {'Title': 'The Chicago History Museum', 'Match': 'Yes', 'URL': 'No URL available'}, {'Title': 'The Museum of Science and Industry', 'Match': 'Yes', 'URL': '

In [ ]:
print(recommendation)

In [58]:
!jupyter nbconvert --to pdf "/content/drive/MyDrive/Colab Notebooks/VideoGeneration_updated.ipynb"


[NbConvertApp] Converting notebook /content/drive/MyDrive/Colab Notebooks/VideoGeneration_updated.ipynb to pdf
[NbConvertApp] Writing 259886 bytes to notebook.tex
[NbConvertApp] Building PDF
Traceback (most recent call last):
  File "/usr/local/bin/jupyter-nbconvert", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/jupyter_core/application.py", line 283, in launch_instance
    super().launch_instance(argv=argv, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.10/dist-packages/nbconvert/nbconvertapp.py", line 423, in start
    self.convert_notebooks()
  File "/usr/local/lib/python3.10/dist-packages/nbconvert/nbconvertapp.py", line 597, in convert_notebooks
    self.convert_single_notebook(notebook_filename)
  File "/usr/local/lib/python3.10/dist-packages/nbconvert/nbconvertapp.py", line 560, in convert_single_notebook
    output,

In [59]:
!jupyter nbconvert --to html "/content/drive/MyDrive/Colab Notebooks/VideoGeneration_updated.ipynb"
!apt-get install -y wkhtmltopdf
!wkhtmltopdf "/content/drive/MyDrive/Colab Notebooks/VideoGeneration_updated.html" "/content/drive/MyDrive/Colab Notebooks/VideoGeneration_updated.pdf"


[NbConvertApp] Converting notebook /content/drive/MyDrive/Colab Notebooks/VideoGeneration_updated.ipynb to html
[NbConvertApp] Writing 896489 bytes to /content/drive/MyDrive/Colab Notebooks/VideoGeneration_updated.html
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  avahi-daemon bind9-host bind9-libs geoclue-2.0 glib-networking glib-networking-common
  glib-networking-services gsettings-desktop-schemas iio-sensor-proxy libavahi-core7 libavahi-glib1
  libdaemon0 libevdev2 libfontenc1 libgudev-1.0-0 libhyphen0 libinput-bin libinput10
  libjson-glib-1.0-0 libjson-glib-1.0-common liblmdb0 libmaxminddb0 libmbim-glib4 libmbim-proxy
  libmd4c0 libmm-glib0 libmtdev1 libnl-genl-3-200 libnotify4 libnss-mdns libproxy1v5 libqmi-glib5
  libqmi-proxy libqt5core5a libqt5dbus5 libqt5gui5 libqt5network5 libqt5positioning5
  libqt5printsupport5 libqt5qml5 libqt5qmlmodels5 libqt5quick5 libqt5sensors5 l

In [57]:
!apt-get install -y texlive-xetex texlive-fonts-recommended texlive-generic-recommended


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package texlive-generic-recommended


In [55]:
!apt-get install -y pandoc


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcmark-gfm-extensions0.29.0.gfm.3 libcmark-gfm0.29.0.gfm.3 pandoc-data
Suggested packages:
  texlive-latex-recommended texlive-xetex texlive-luatex pandoc-citeproc texlive-latex-extra
  context wkhtmltopdf librsvg2-bin groff ghc nodejs php python ruby libjs-mathjax libjs-katex
  citation-style-language-styles
The following NEW packages will be installed:
  libcmark-gfm-extensions0.29.0.gfm.3 libcmark-gfm0.29.0.gfm.3 pandoc pandoc-data
0 upgraded, 4 newly installed, 0 to remove and 49 not upgraded.
Need to get 20.6 MB of archives.
After this operation, 156 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libcmark-gfm0.29.0.gfm.3 amd64 0.29.0.gfm.3-3 [115 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libcmark-gfm-extensions0.29.0.gfm.3 amd64 0.29.0.gfm.3-3 [25.1 kB]


In [53]:
!apt-get install -y texlive-xetex texlive-fonts-recommended texlive-generic-recommended
!pip install nbconvert


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package texlive-generic-recommended
